# Data Preprocessing - Neater Implementation
___

In [2]:
# Enable Intellisense
%config IPCompleter.greedy=True

import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
from matplotlib import pyplot, dates
import plotly.express as px 
import datetime
import plotly.graph_objects as go
import plotly.io as pio
sns.set()

# Annoying warning
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# ['EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW', 'WC']

## Gen Helper Functions

In [4]:
def df_from_url(df_url, pd_kwargs={}):
    df_req = requests.get(df_url).content
    df = pd.read_csv(io.StringIO(df_req.decode('utf-8')), **pd_kwargs)
    return df

In [5]:
tmp_kwargs = {"usecols":['date','total']}
df_from_url("https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv", 
            tmp_kwargs).head()

,date,total
0,05-03-2020,1
1,07-03-2020,2
2,08-03-2020,3
3,09-03-2020,7
4,11-03-2020,13


# Cumulative/Daily Totals Per Day
Currently only applicable for confirmed cases and deaths.

In [6]:
def get_cum_daily(data_url, cum_col='total', index_col='date'): # kwargs={}):
#     data_req = requests.get(data_url).content
#     data = pd.read_csv(io.StringIO(data_req.decode('utf-8')), usecols=cols, index_col=['date'])
    cols = ['date', 'total']
    pd_kwargs = {"usecols":[cum_col, index_col],"index_col":[index_col]}
#     if usecols != []:
#         pd_kwargs.update({"usecols":usecols})
#     pd_kwargs.update(kwargs)
    data = df_from_url(data_url, pd_kwargs)
    data.reset_index(inplace=True)
    data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y')
    data.set_index('date', inplace = True)
    data.rename({cum_col:"cum_no"}, axis=1, inplace = True)
    data.ffill(inplace=True)
    
    data['daily_no'] = data['cum_no']
    # slice is required as first entry of diff will be NaN but data we want should be equal to the 
    # starting value
    data['daily_no'][1:] = data['cum_no'].diff()[1:]
    # Cast columns to integer
    data = data.astype('int32')
    return data

## Confirmed Cases

In [7]:
confirmed_cases_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv"
confirmed_data = get_cum_daily(confirmed_cases_url)
confirmed_data

,cum_no,daily_no
date,,
2020-03-05,1,1
2020-03-07,2,1
2020-03-08,3,1
2020-03-09,7,4
2020-03-11,13,6
...,...,...
2020-05-08,8895,663
2020-05-09,9420,525
2020-05-10,10015,595


## Deaths

In [8]:
deaths_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_deaths.csv"
deaths_data = get_cum_daily(deaths_url)
deaths_data

,cum_no,daily_no
date,,
2020-03-27,1,1
2020-03-28,2,1
2020-03-30,3,1
2020-03-31,5,2
2020-04-03,9,4
2020-04-05,11,2
2020-04-06,12,1
2020-04-07,13,1
2020-04-08,18,5


## Tests

In [9]:
tests_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv"
tests_data = get_cum_daily(tests_url, 'cumulative_tests', 'date')
tests_data

,cum_no,daily_no
date,,
2020-02-11,61,61
2020-02-13,67,6
2020-02-14,71,4
2020-02-19,95,24
2020-02-20,106,11
...,...,...
2020-05-08,307752,15599
2020-05-09,324079,16327
2020-05-10,341336,17257


## Recoveries

In [10]:
tests_url = "https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_timeline_testing.csv"
recovered_data = get_cum_daily(tests_url, 'recovered', 'date')
recovered_data

,cum_no,daily_no
date,,
2020-02-11,0,0
2020-02-13,0,0
2020-02-14,0,0
2020-02-19,0,0
2020-02-20,0,0
...,...,...
2020-05-08,3153,0
2020-05-09,3983,830
2020-05-10,4173,190


## Active Cases
Cases that have not yet had an outcome. I.e. confirmed cases - recovered - deaths

In [11]:
active_data = confirmed_data[['cum_no']].copy().rename({"cum_no":"confirmed"}, axis = 1)
active_data = pd.concat([active_data, 
                         recovered_data[['cum_no']].copy().rename({"cum_no":"recovered"}, axis =1),
                         deaths_data[['cum_no']].copy().rename({"cum_no":"deaths"}, axis = 1)
                        ], 
                        axis =1)
# active_data.fillna(0, inplace=True)
active_data = active_data.iloc[9:]
active_data = active_data.ffill().fillna(0)
active_data

,confirmed,recovered,deaths
date,,,
2020-03-05,1.0,0.0,0.0
2020-03-06,1.0,0.0,0.0
2020-03-07,2.0,0.0,0.0
2020-03-08,3.0,0.0,0.0
2020-03-09,7.0,0.0,0.0
...,...,...,...
2020-05-08,8895.0,3153.0,178.0
2020-05-09,9420.0,3983.0,186.0
2020-05-10,10015.0,4173.0,194.0


In [12]:
active_data['cum_no'] = active_data['confirmed'] - active_data['recovered'] - active_data['deaths']
active_data.drop(['confirmed','recovered','deaths'], axis=1, inplace=True)
active_data['daily_no'] = active_data['cum_no'].copy()
active_data['daily_no'].iloc[1:] = active_data['cum_no'].diff().iloc[1:]
active_data = active_data.astype('int32')
active_data

,cum_no,daily_no
date,,
2020-03-05,1,1
2020-03-06,1,0
2020-03-07,2,1
2020-03-08,3,1
2020-03-09,7,4
...,...,...
2020-05-08,5564,646
2020-05-09,5251,-313
2020-05-10,5648,397


## All Cumulative/Totals Per Day

In [13]:
all_cum_data = confirmed_data[['cum_no']].rename({"cum_no":"confirmed"}, axis =1)
all_cum_data = pd.concat([
    all_cum_data, 
    tests_data[['cum_no']].rename({"cum_no":"tests"},axis=1),
    deaths_data[['cum_no']].rename({"cum_no":"deaths"},axis=1),
    recovered_data[['cum_no']].rename({"cum_no":"recovered"},axis=1),
    active_data[['cum_no']].rename({"cum_no":"active"},axis=1),

], axis=1)
# all_cum_data['recovered'] = recovered_data['cum_no']
# all_cum_data['active'] = active_data['cum_no']
all_cum_data.ffill(inplace=True)
all_cum_data.fillna(0, inplace=True)
all_cum_data = all_cum_data.astype('int32')
all_cum_data

,confirmed,tests,deaths,recovered,active
date,,,,,
2020-02-11,0,61,0,0,0
2020-02-13,0,67,0,0,0
2020-02-14,0,71,0,0,0
2020-02-19,0,95,0,0,0
2020-02-20,0,106,0,0,0
...,...,...,...,...,...
2020-05-08,8895,307752,178,3153,5564
2020-05-09,9420,324079,186,3983,5251
2020-05-10,10015,341336,194,4173,5648


## Derived Stats
Added to cumulative/totals per day `all_cum_data`
### Confirmed divided by Tests

In [14]:
all_cum_data['confirmed_div_by_tests'] = all_cum_data['confirmed']/all_cum_data['tests']
all_cum_data['confirmed_div_by_tests'] = all_cum_data['confirmed_div_by_tests'].round(3)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests
date,,,,,,
2020-02-11,0,61,0,0,0,0.000
2020-02-13,0,67,0,0,0,0.000
2020-02-14,0,71,0,0,0,0.000
2020-02-19,0,95,0,0,0,0.000
2020-02-20,0,106,0,0,0,0.000
...,...,...,...,...,...,...
2020-05-08,8895,307752,178,3153,5564,0.029
2020-05-09,9420,324079,186,3983,5251,0.029
2020-05-10,10015,341336,194,4173,5648,0.029


### Deaths divided by confirmed

In [15]:
all_cum_data['deaths_div_by_confirmed'] = all_cum_data['deaths']/all_cum_data['confirmed']
all_cum_data['deaths_div_by_confirmed'] = all_cum_data['deaths_div_by_confirmed'].round(3)
all_cum_data.fillna(0.000, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed
date,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000
2020-02-13,0,67,0,0,0,0.000,0.000
2020-02-14,0,71,0,0,0,0.000,0.000
2020-02-19,0,95,0,0,0,0.000,0.000
2020-02-20,0,106,0,0,0,0.000,0.000
...,...,...,...,...,...,...,...
2020-05-08,8895,307752,178,3153,5564,0.029,0.020
2020-05-09,9420,324079,186,3983,5251,0.029,0.020
2020-05-10,10015,341336,194,4173,5648,0.029,0.019


### Recovered divided by confirmed

In [16]:
all_cum_data['recovered_div_by_confirmed'] = all_cum_data['recovered']/all_cum_data['confirmed']
all_cum_data['recovered_div_by_confirmed'] = all_cum_data['recovered_div_by_confirmed'].round(3)
all_cum_data.fillna(0.000, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed,recovered_div_by_confirmed
date,,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000,0.000
2020-02-13,0,67,0,0,0,0.000,0.000,0.000
2020-02-14,0,71,0,0,0,0.000,0.000,0.000
2020-02-19,0,95,0,0,0,0.000,0.000,0.000
2020-02-20,0,106,0,0,0,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...
2020-05-08,8895,307752,178,3153,5564,0.029,0.020,0.354
2020-05-09,9420,324079,186,3983,5251,0.029,0.020,0.423
2020-05-10,10015,341336,194,4173,5648,0.029,0.019,0.417


### Stats Per Million Population
[Source for stats](https://worldpopulationreview.com/countries/south-africa-population/)

In [17]:
sa_tot_population = 59195720
# total population rounded in millions
sa_tot_pop_mil = sa_tot_population/1000000
sa_tot_pop_mil

59.19572

In [18]:
all_cum_data['confirmed_per_mil'] = all_cum_data['confirmed']/sa_tot_pop_mil
all_cum_data['tests_per_mil'] = all_cum_data['tests']/sa_tot_pop_mil
all_cum_data['deaths_per_mil'] = all_cum_data['deaths']/sa_tot_pop_mil
all_cum_data['recovered_per_mil'] = all_cum_data['recovered']/sa_tot_pop_mil
all_cum_data['active_per_mil'] = all_cum_data['active']/sa_tot_pop_mil
tmp_cols = ['confirmed_per_mil','tests_per_mil','deaths_per_mil','recovered_per_mil','active_per_mil']
all_cum_data[tmp_cols] = all_cum_data[tmp_cols].round(2)
all_cum_data.fillna(0.00, inplace=True)
all_cum_data

,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed,recovered_div_by_confirmed,confirmed_per_mil,tests_per_mil,deaths_per_mil,recovered_per_mil,active_per_mil
date,,,,,,,,,,,,,
2020-02-11,0,61,0,0,0,0.000,0.000,0.000,0.00,1.03,0.00,0.00,0.00
2020-02-13,0,67,0,0,0,0.000,0.000,0.000,0.00,1.13,0.00,0.00,0.00
2020-02-14,0,71,0,0,0,0.000,0.000,0.000,0.00,1.20,0.00,0.00,0.00
2020-02-19,0,95,0,0,0,0.000,0.000,0.000,0.00,1.60,0.00,0.00,0.00
2020-02-20,0,106,0,0,0,0.000,0.000,0.000,0.00,1.79,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-08,8895,307752,178,3153,5564,0.029,0.020,0.354,150.26,5198.89,3.01,53.26,93.99
2020-05-09,9420,324079,186,3983,5251,0.029,0.020,0.423,159.13,5474.70,3.14,67.29,88.71
2020-05-10,10015,341336,194,4173,5648,0.029,0.019,0.417,169.18,5766.23,3.28,70.49,95.41


**Save to csv**

In [19]:
all_cum_data.to_csv('data/all_cum_data.csv')

## All Daily Change Data Per Day

In [20]:
all_daily_data = confirmed_data[['daily_no']].rename({"daily_no":"confirmed"}, axis =1)
all_daily_data = pd.concat([
    all_daily_data, 
    tests_data[['daily_no']].rename({"daily_no":"tests"},axis=1),
    deaths_data[['daily_no']].rename({"daily_no":"deaths"},axis=1),
    recovered_data[['daily_no']].rename({"daily_no":"recovered"},axis=1),
    active_data[['daily_no']].rename({"daily_no":"active"},axis=1),

], axis=1)
# all_daily_data['recovered'] = recovered_data['daily_no']
# all_daily_data['active'] = active_data['daily_no']
all_daily_data.ffill(inplace=True)
all_daily_data.fillna(0, inplace=True)
all_daily_data = all_daily_data.astype('int32')
all_daily_data

,confirmed,tests,deaths,recovered,active
date,,,,,
2020-02-11,0,61,0,0,0
2020-02-13,0,6,0,0,0
2020-02-14,0,4,0,0,0
2020-02-19,0,24,0,0,0
2020-02-20,0,11,0,0,0
...,...,...,...,...,...
2020-05-08,663,15599,17,0,646
2020-05-09,525,16327,8,830,-313
2020-05-10,595,17257,8,190,397


**Save to csv**

In [21]:
all_daily_data.to_csv("data/all_daily_data.csv")

### Change format of date

In [22]:
all_cum_data_alt = all_cum_data.copy().reset_index()

# Daily/Cumulative Data Per Prov Per Day

In [23]:
def get_cum_daily_by_prov(data_url):
    cols = ['date','EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW','WC', 'UNKNOWN']
    pd_kwargs = {"usecols":cols}
    cum_data = df_from_url(data_url, pd_kwargs)
    cum_data.dropna(inplace=True)
    cum_data['date'] = pd.to_datetime(cum_data['date'], format='%d-%m-%Y')
    
    daily_data = cum_data.copy()
    daily_data.iloc[1:,1:] = daily_data.iloc[:,1:].diff().iloc[1:]
    daily_data_melt = daily_data.melt(id_vars=['date'], var_name='province', value_name='daily_no')
    daily_data_melt.set_index(['date'], inplace=True)
    
    
    cum_data_melt = cum_data.melt(id_vars=['date'], var_name='province', value_name='cum_no')
    cum_data_melt.set_index(['date'], inplace=True)
    
    data = pd.concat([cum_data_melt, daily_data_melt[['daily_no']]], axis = 1)
    data[['cum_no','daily_no']] = data[['cum_no','daily_no']].astype('int32')

    return data

In [24]:
# daily_data.iloc[1:,1:] = daily_data.iloc[:,1:].diff().iloc[1:]
# daily_data

## Confirmed Cases Per Prov
Daily change and cumulative

In [25]:
confirmed_by_prov_timeline = get_cum_daily_by_prov("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_confirmed.csv")
confirmed_by_prov_timeline

,province,cum_no,daily_no
date,,,
2020-03-05,EC,0,0
2020-03-07,EC,0,0
2020-03-08,EC,0,0
2020-03-09,EC,0,0
2020-03-11,EC,0,0
...,...,...,...
2020-05-08,UNKNOWN,0,0
2020-05-09,UNKNOWN,0,0
2020-05-10,UNKNOWN,0,0


**Save to csv**

In [26]:
confirmed_by_prov_timeline.to_csv("data/confirmed_by_prov_timeline.csv")

## Deaths Per Prov
Daily change and cumulative

In [27]:
deaths_by_prov_timeline = get_cum_daily_by_prov("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_deaths.csv")
deaths_by_prov_timeline

,province,cum_no,daily_no
date,,,
2020-03-27,EC,0,0
2020-03-28,EC,0,0
2020-03-30,EC,0,0
2020-03-31,EC,0,0
2020-04-03,EC,0,0
...,...,...,...
2020-05-07,UNKNOWN,0,0
2020-05-08,UNKNOWN,0,0
2020-05-09,UNKNOWN,0,0


**Save to csv**

In [28]:
deaths_by_prov_timeline.to_csv("data/deaths_by_prov_timeline.csv")

# Total & Latest Change

In [29]:
def get_tot_latest_change(data_url, ):
    cols = ['date','EC', 'FS', 'GP', 'KZN', 'LP', 'MP', 'NC', 'NW','WC', 'UNKNOWN']
    pd_kwargs = {"usecols":cols}
    cum_data = df_from_url(data_url, pd_kwargs)
    cum_data.dropna(inplace=True)
    cum_data['date'] = pd.to_datetime(cum_data['date'], format='%d-%m-%Y')
    
    province_names = {
        "EC":"Eastern Cape",
        "FS" : "Free State",
        "GP" : "Gauteng",
        "KZN" : "KwaZulu-Natal",
        "LP" : "Limpopo",
        "MP" : "Mpumalanga",
        "NW" : "North West",
        "NC" : "Northern Cape",
        "WC" : "Western Cape",
        "UNKNOWN": "Unknown"
    }
    
    daily_data = cum_data.copy()
    daily_data.iloc[1:,1:] = daily_data.iloc[:,1:].diff().iloc[1:]
    daily_data = daily_data.tail(1) # get last entry
    daily_data_melt = daily_data.melt(id_vars=['date'], var_name='province', value_name='latest_change')
    daily_data_melt['province'] = daily_data_melt['province'].map(province_names)
    daily_data_melt.set_index(['province'], inplace=True)
    
    cum_data = cum_data.tail(1) # get last entry
    cum_data_melt = cum_data.melt(id_vars=['date'], var_name='province', value_name='total')
    cum_data_melt['province'] = cum_data_melt['province'].map(province_names)
    cum_data_melt.set_index(['province'], inplace=True)
    
    data = pd.concat([cum_data_melt, daily_data_melt[['latest_change']]], axis = 1)
    data.drop(['date'], axis=1, inplace=True)
    data = data.astype('int32')

    return data

## Deaths

In [30]:
deaths_by_prov_total = get_tot_latest_change("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_deaths.csv")
deaths_by_prov_total

,total,latest_change
province,,
Eastern Cape,24,2
Free State,6,0
Gauteng,22,0
KwaZulu-Natal,44,1
Limpopo,3,0
Mpumalanga,0,0
Northern Cape,0,0
North West,1,1
Western Cape,106,8


**Save to csv**

In [31]:
deaths_by_prov_total.to_csv('data/tot_deaths_provinces.csv')

## Confirmed

In [32]:
confirmed_by_prov_total = get_tot_latest_change("https://raw.githubusercontent.com/dsfsi/covid19za/master/" +
                      "data/covid19za_provincial_cumulative_timeline_confirmed.csv")
confirmed_by_prov_total

,total,latest_change
province,,
Eastern Cape,1504,148
Free State,135,0
Gauteng,2014,43
KwaZulu-Natal,1394,22
Limpopo,54,0
Mpumalanga,63,0
Northern Cape,30,0
North West,51,1
Western Cape,6105,484


**Save to csv**

In [33]:
confirmed_by_prov_total.to_csv('data/tot_provinces.csv')

## Data to be added to text of website

In [34]:
def zero_space(num):
    return format(num,',d').replace(","," ")

In [35]:
tests_data

,cum_no,daily_no
date,,
2020-02-11,61,61
2020-02-13,67,6
2020-02-14,71,4
2020-02-19,95,24
2020-02-20,106,11
...,...,...
2020-05-08,307752,15599
2020-05-09,324079,16327
2020-05-10,341336,17257


In [36]:
tot_tested = zero_space(tests_data.tail(1).iloc[0]['cum_no'].astype(int))
change_tested = zero_space(tests_data.tail(1).iloc[0]['daily_no'].astype(int))
print(tot_tested + ", " + change_tested)

369 697, 13 630


In [37]:
tot_infected = zero_space(confirmed_data.tail(1).iloc[0]['cum_no'].astype(int))
change_infected = zero_space(confirmed_data.tail(1).iloc[0]['daily_no'].astype(int))
print(tot_infected + ", " + change_infected)

11 350, 698


In [38]:
tot_deaths = zero_space(deaths_data.tail(1).iloc[0]['cum_no'].astype(int))
change_deaths = zero_space(deaths_data['daily_no'].tail(1).iloc[0].astype(int))
print(tot_deaths + ", " + change_deaths)

206, 12


In [39]:
tot_recoveries = zero_space(recovered_data.tail(1).iloc[0]['cum_no'].astype(int))
change_recoveries = zero_space(recovered_data.tail(1).iloc[0]['daily_no'].astype(int))
print(tot_recoveries + ", " + change_recoveries)

4 357, 0


In [40]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M %d %B %Y")
print("Current Time =", current_time)

Current Time = 15:12 13 May 2020


In [41]:
gen_data = pd.DataFrame(dict(tot_infected=[tot_infected], change_infected=[change_infected], 
                             tot_deaths=[tot_deaths], change_deaths=[change_deaths],
                             tot_tested=[tot_tested], change_tested=[change_tested], 
                             tot_recoveries=[tot_recoveries], change_recoveries=[change_recoveries], 
                             datetime_updated=[current_time]))
gen_data

,tot_infected,change_infected,tot_deaths,change_deaths,tot_tested,change_tested,tot_recoveries,change_recoveries,datetime_updated
0,11 350,698,206,12,369 697,13 630,4 357,0,15:12 13 May 2020


In [42]:
gen_data.to_csv("data/gen_data.csv", index=False)

### Render Template

In [43]:
import template_renderer as tr
tr.render_all()

Template Rendered - 15:12 13 May 2020
Template Rendered - 15:12 13 May 2020
